### PyAudio test

In [1]:
import time

import numpy as np
import pyaudio

from matplotlib import pyplot as plt

p = pyaudio.PyAudio()

volume = 0.5  # range [0.0, 1.0]
fs = 44100  # sampling rate, Hz, must be integer
duration = 5.0  # in seconds, may be float
f = 440.0  # sine frequency, Hz, may be float

# generate samples, note conversion to float32 array
samples = (np.sin(2 * np.pi * np.arange(fs * duration) * f / fs)).astype(np.float32)

# per @yahweh comment explicitly convert to bytes sequence
output_bytes = (volume * samples).tobytes()

# for paFloat32 sample values must be in range [-1.0, 1.0]
stream = p.open(format=pyaudio.paFloat32,
                channels=1,
                rate=fs,
                output=True)

# play. May repeat with different volume values (if done interactively)
start_time = time.time()
stream.write(output_bytes)
print("Played sound for {:.2f} seconds".format(time.time() - start_time))

stream.stop_stream()
stream.close()

p.terminate()

Played sound for 5.01 seconds


### Generate a piano note

In [8]:
p = pyaudio.PyAudio()

volume = 0.5  # range [0.0, 1.0]
fs = 44100  # sampling rate, Hz, must be integer
duration = 3.0  # in seconds, may be float

f_fundamental = 440.0  # sine frequency, Hz, may be float
spectrum_index = -1.0 # index of the frequency in the spectrum
exponential_factor = -0.15
switch_to_exp_mode = 9
nmodes = 12

volume_decay_time = 1.5
attack_time = 0.01

abscissa = np.arange(fs * duration)
samples = np.zeros_like(abscissa, dtype=np.float32)

for imode in range(1,nmodes+1):
    f = f_fundamental * (imode)
    mode = np.sin(2 * np.pi * abscissa * f / fs)
    
    if imode < switch_to_exp_mode:
        amplitude = imode**spectrum_index
    else:
        amplitude = ((switch_to_exp_mode-1)**spectrum_index) * np.exp(exponential_factor * (imode-switch_to_exp_mode+1))
    print(imode, amplitude)

    samples += amplitude * mode * np.exp(-imode**1.5*abscissa/(volume_decay_time*fs))

# decay and attack
#samples *= np.exp(-abscissa/(volume_decay_time*fs))
#samples *= np.tanh(abscissa/(attack_time * fs))

# per @yahweh comment explicitly convert to bytes sequence
output_bytes = (volume * samples).tobytes()

# for paFloat32 sample values must be in range [-1.0, 1.0]
stream = p.open(format=pyaudio.paFloat32,
                channels=1,
                rate=fs,
                output=True)

# play. May repeat with different volume values (if done interactively)
start_time = time.time()
stream.write(output_bytes)
print("Played sound for {:.2f} seconds".format(time.time() - start_time))

stream.stop_stream()
stream.close()

p.terminate()

1 1.0
2 0.5
3 0.3333333333333333
4 0.25
5 0.2
6 0.16666666666666666
7 0.14285714285714285
8 0.125
9 0.10758849705313223
10 0.09260227758521473
11 0.07970351895272167
12 0.0686014545117533
Played sound for 3.00 seconds


In [35]:
def define_abscissa(fs=44100, duration=1.0):
    return np.arange(fs * duration)

def generate_one_note(f_fundamental, abscissa, 
                      spectrum_index=-1.0, exponential_factor=-0.15, switch_to_exp_mode=9, nmodes=12,
                      fs=44100, decay_exponent_frequency=1.,
                      volume_decay_time=2.5, attack_time=0.015,
                      inharmonicity=0.0):
    samples = np.zeros_like(abscissa, dtype=np.float32)
    for imode in range(1,nmodes+1):
        f = f_fundamental * (imode) * (1 + inharmonicity * (imode-1)**2 / 2)
        mode = np.sin(2 * np.pi * abscissa * f / fs)

        if imode < switch_to_exp_mode:
            amplitude = imode**spectrum_index
        else:
            amplitude = ((switch_to_exp_mode-1)**spectrum_index) * np.exp(exponential_factor * (imode-switch_to_exp_mode+1))
        #print(imode, amplitude)

        samples += amplitude * mode * np.exp(-imode**decay_exponent_frequency*abscissa/(volume_decay_time*fs))
        
    # decay and attack
    #samples *= np.exp(-abscissa/(volume_decay_time*fs))
    samples *= np.tanh(abscissa/(attack_time * fs))

    return samples


def play_samples(samples, volume=0.5, fs=44100):
    output_bytes = (volume * samples).tobytes()
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paFloat32,
                    channels=1,
                    rate=fs,
                    output=True)
    start_time = time.time()
    stream.write(output_bytes)
    print("Played sound for {:.2f} seconds".format(time.time() - start_time))
    stream.stop_stream()
    stream.close()
    p.terminate()


In [60]:
abscissa = define_abscissa(duration=5.0)
inharmonicity = 0.001
play_samples(generate_one_note(220.0, abscissa, inharmonicity=inharmonicity) + generate_one_note(441.7, abscissa, inharmonicity=inharmonicity))

Played sound for 5.00 seconds


In [54]:
# print a table with the first 10 harmonics
inharmonicity = 0.001
f1 = 220. 
f2 = 441.7
for imode in range(1,11):
    fh1 = f1 * imode * (1 + inharmonicity * (imode-1)**2 / 2)
    fh2 = f2 * imode * (1 + inharmonicity * (imode-1)**2 / 2)
    print('{:} \t {:.2f} \t {:.2f}'.format(imode, fh1, fh2))

1 	 220.00 	 441.70
2 	 440.22 	 883.84
3 	 661.32 	 1327.75
4 	 883.96 	 1774.75
5 	 1108.80 	 2226.17
6 	 1336.50 	 2683.33
7 	 1567.72 	 3147.55
8 	 1803.12 	 3620.17
9 	 2043.36 	 4102.51
10 	 2289.10 	 4595.89


In [49]:
abscissa = define_abscissa(duration=5.0)
inharmonicity = 0.001
play_samples(generate_one_note(444.61, abscissa, inharmonicity=inharmonicity) + generate_one_note(220.0, abscissa, inharmonicity=inharmonicity))

Played sound for 5.01 seconds
